# **Simple Scenario Example - Charleston, USA**

In this notebook we demonstrate the **workflow** on how you can build a simple **FloodAdapt scenario** in Charleston, USA, using the **API**.  

We prepared a database, which you can download from [**here**](link.com). If you worked with the [**Database-Builder notebook**](database_builder.ipynb) beforehand, you can use that database.


In this notebook we will cover the following steps:

1. Create a synthetic **event** - Rainfall
2. Create a **projection** - Sea level rise (SLR)
3. Create a **measure** and **strategy** - Seawall
4. Create and run a **scenario**
6. Investigate the **output**


## **Step 0**: Import packages

In [1]:
# Import packages
from datetime import datetime
from pathlib import Path
from flood_adapt import Settings
from flood_adapt.dbs_classes.database import Database
from flood_adapt import unit_system as us
from flood_adapt.flood_adapt import FloodAdapt
from flood_adapt.objects.events.synthetic import SyntheticEvent

from flood_adapt.objects.forcing.forcing import (
    ForcingSource,
    ForcingType,
)
from flood_adapt.objects.forcing.time_frame import (
    TimeFrame,
)
from flood_adapt.objects.forcing.timeseries import (
    ShapeType,
    TimeseriesFactory,
)
from flood_adapt.config.sfincs import RiverModel
from flood_adapt.objects.forcing.discharge import DischargeConstant
from flood_adapt.objects.forcing.forcing import ForcingType
from flood_adapt.objects.forcing.rainfall import RainfallConstant
from flood_adapt.objects.forcing.waterlevels import (
    SurgeModel,
    TideModel,
    WaterlevelSynthetic,
)
from flood_adapt.objects.forcing.wind import WindConstant

## **Step 1**: Getting started - Initiate the database

Let's start with initiating the database.

In [2]:
# Define the full path to your database file
full_path = Path(r"C:\Users\rautenba\Local_FA_Databases\Database\charleston_full")

# Set up the settings for the database
Settings(DATABASE_ROOT=full_path.parent, DATABASE_NAME=full_path.name)

# Create the FloodAdapt instance
fa = FloodAdapt(Settings().database_path)

## **Step 2**: Events - Create a synthetic Event

In [ ]:
attrs = SyntheticEvent(
        name="test_synthetic_nearshore",
        time=TimeFrame(
            start_time=datetime(2020, 1, 1),
            end_time=datetime(2020, 1, 2),
        ),
        forcings={
            ForcingType.WIND: [
                WindConstant(
                    speed=us.UnitfulVelocity(value=5, units=us.UnitTypesVelocity.mps),
                    direction=us.UnitfulDirection(
                        value=60, units=us.UnitTypesDirection.degrees
                    ),
                )
            ],
            ForcingType.RAINFALL: [
                RainfallConstant(
                    intensity=us.UnitfulIntensity(
                        value=20, units=us.UnitTypesIntensity.mm_hr
                    )
                )
            ],
            ForcingType.DISCHARGE: [
                DischargeConstant(
                    river=RiverModel(
                        name="cooper",
                        description="Cooper River",
                        x_coordinate=595546.3,
                        y_coordinate=3675590.6,
                        mean_discharge=us.UnitfulDischarge(
                            value=5000, units=us.UnitTypesDischarge.cfs
                        ),
                    ),
                    discharge=us.UnitfulDischarge(
                        value=5000, units=us.UnitTypesDischarge.cfs
                    ),
                )
            ],
            ForcingType.WATERLEVEL: [
                WaterlevelSynthetic(
                    surge=SurgeModel(
                        timeseries=TimeseriesFactory.from_args(
                            shape_type=ShapeType.triangle,
                            duration=us.UnitfulTime(
                                value=1, units=us.UnitTypesTime.days
                            ),
                            peak_time=us.UnitfulTime(
                                value=8, units=us.UnitTypesTime.hours
                            ),
                            peak_value=us.UnitfulLength(
                                value=1, units=us.UnitTypesLength.meters
                            ),
                        )
                    ),
                    tide=TideModel(
                        harmonic_amplitude=us.UnitfulLength(
                            value=1, units=us.UnitTypesLength.meters
                        ),
                        harmonic_period=us.UnitfulTime(
                            value=12.4, units=us.UnitTypesTime.hours
                        ),
                        harmonic_phase=us.UnitfulTime(
                            value=0, units=us.UnitTypesTime.hours
                        ),
                    ),
                )
            ],
        },
    )

synthetic = fa.create_event(attrs.dict())
fa.save_event(synthetic) #not sure if hat is necessary

## **Step 3**: Projections - Create a projection

## **Step X**: Create and run a scenario

In [8]:
event_name = synthetic.name
scenario_dict = {
    "name": "current_nearshore_no_measures",
    "description": "current_nearshore_no_measures",
    "event": event_name,
    "projection": "current",
    "strategy": "no_measures",
}
scenario = fa.create_scenario(scenario_dict)
fa.save_scenario(scenario)

fa.run_scenario(scenario.name)

2025-05-07 10:51:28 AM - FloodAdapt.SfincsAdapter - WARNING - Failed to add event rainfall multiplier, no rainfall forcing found in the model.
